In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
chunked_data = pd.read_csv('src_data/cbg2cbg_revise.csv', chunksize=100000)
chunk_list = []
for chunk in chunked_data:
    chunk_list.append(chunk)
transport_data = pd.concat(chunk_list)

In [3]:
group_by_src = transport_data.groupby('poi_cbg_source')
group_by_des = transport_data.groupby('poi_cbg_destination')

In [4]:
group_by_des.get_group(11010060001).sum()

poi_cbg_source         633275749573
poi_cbg_destination    660603600060
src_prob                        1.0
des_prob                   0.260424
dtype: object

In [11]:
src_cbgs = list(transport_data['poi_cbg_source'])

In [16]:
test_df = group_by_src.get_group(src_cbgs[0])

In [17]:
test_df.sum()

poi_cbg_source         380387638038
poi_cbg_destination    401385212357
src_prob                   0.288262
des_prob                        1.0
dtype: object

In [18]:
probs = {}
des_cbgs = list(test_df['poi_cbg_destination'])
des_probs = list(test_df['des_prob'])
src_cbg = test_df['poi_cbg_source'].iloc[0]
for des_cbg, des_prob in zip(des_cbgs, des_probs):
    des_prob_df = group_by_des.get_group(des_cbg)
    rev_src_cbg = list(des_prob_df['poi_cbg_source'])
    rev_src_prob = list(des_prob_df['src_prob'])
    for c,p in zip(rev_src_cbg,rev_src_prob):
        if c not in probs: probs[c] = 0
        probs[c] += des_prob * p

probs = pd.DataFrame(probs.items(), columns = ['to', 'prob'])
probs['from'] = len(probs) * [src_cbg]
probs = probs[['from', 'to', 'prob']]
probs

,from,to,prob
0,10010201001,10010201001,0.010348
1,10010201001,10010201002,0.016691
2,10010201001,10010202001,0.009565
3,10010201001,10010202002,0.012180
4,10010201001,10010203001,0.032125
...,...,...,...
1449,10010201001,11170303041,0.000019
1450,10010201001,11170303053,0.000019
1451,10010201001,11170303303,0.000019
1452,10010201001,120910215021,0.000019


In [9]:
probs.sum()

0    1.178923e+14
1    1.000000e+00
dtype: float64

In [ ]:
des_prob_df.sum()

In [11]:
group_by_des.get_group(11010060001)

,poi_cbg_source,poi_cbg_destination,src_prob,des_prob
37,10010201001,11010060001,0.011811,0.006952
84,10010201002,11010060001,0.017717,0.005039
130,10010202001,11010060001,0.011811,0.006179
182,10010202002,11010060001,0.011811,0.004454
269,10010203001,11010060001,0.021654,0.002450
...,...,...,...,...
257233,11250112001,11010060001,0.015748,0.000123
257759,11250114013,11010060001,0.005906,0.000557
259007,11250120003,11010060001,0.005906,0.000538
14151749,470370103012,11010060001,0.500000,0.000515


In [12]:
group_by_des.get_group(11010060001).sum()

poi_cbg_source         1575296061608
poi_cbg_destination     682623720062
src_prob                         2.0
des_prob                     0.26153
dtype: object

In [19]:
def cal_spread_probs(src_cbg_df):
    probs = {}
    des_cbgs = list(src_cbg_df['poi_cbg_destination'])
    des_probs = list(src_cbg_df['des_prob'])
    src_cbg = src_cbg_df['poi_cbg_source'].iloc[0]
    for des_cbg, des_prob in zip(des_cbgs, des_probs):
        des_prob_df = group_by_des.get_group(des_cbg)
        rev_src_cbg = list(des_prob_df['poi_cbg_source'])
        rev_src_prob = list(des_prob_df['src_prob'])
        for c,p in zip(rev_src_cbg,rev_src_prob):
            if c not in probs: probs[c] = 0
            probs[c] += des_prob * p

    probs = pd.DataFrame(probs.items(), columns = ['to', 'prob'])
    probs['from'] = len(probs) * [src_cbg]
    probs = probs[['from', 'to', 'prob']]
    return probs

In [ ]:
results = group_by_src.apply(cal_spread_probs)

In [19]:
import pandas as pd

df = pd.DataFrame({
    'group': ['A', 'A', 'B', 'B'],
    'value1': [1, 2, 3, 4],
    'value2': [5, 6, 7, 8]
})

def calculate_stats(group):
    r = {}
    r['g'] = list(group['group'])[:2]
    r['value1'] = list(group['value1'])[:2]
    r['value2'] = list(group['value1'])[:2]

    probs = pd.DataFrame(r)
    return probs


result = df.groupby('group')[df.columns.tolist()].apply(calculate_stats).reset_index(drop=True)

print(result)

   g  value1  value2
0  A       1       1
1  A       2       2
2  B       3       3
3  B       4       4
